In [1]:
import pandas as pd
import numpy as np

In [2]:
# Завантаження та огляд даних 
df = pd.read_csv(r'C:\Users\PC\Desktop\datasets\bank-full.csv', sep = ';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [4]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [5]:
df.shape

(45211, 17)

In [6]:
# Загальна конверсія людей які погодилися на депозит
df['y'].value_counts(normalize=True)

y
no     0.883015
yes    0.116985
Name: proportion, dtype: float64

In [7]:
# Детальні статистичні показники тривалості успішних дзвінків
# (середнє, медіана, мінімум, максимум та квартилі)
success = df[df['y'] == 'yes']
success['duration'].describe()

count    5289.000000
mean      537.294574
std       392.525262
min         8.000000
25%       244.000000
50%       426.000000
75%       725.000000
max      3881.000000
Name: duration, dtype: float64

In [8]:
# Порівнюємо середній баланс клієнтів які погодилися та відмовилися
df.groupby('y')['balance'].mean()

y
no     1303.714969
yes    1804.267915
Name: balance, dtype: float64

In [9]:
# Розрахунок відносного рівня конверсії залежно від рівня освіти клієнта у відсотках
pd.crosstab(df['education'], df['y'], normalize = 'index')

y,no,yes
education,,
primary,0.913735,0.086265
secondary,0.894406,0.105594
tertiary,0.849936,0.150064
unknown,0.864297,0.135703


In [10]:
# Таблицю спряженості для аналізу конверсії за професіями
job_y = pd.crosstab(df['job'], df['y'], normalize = 'index')
job_y.sort_values(by = 'yes', ascending = False)

y,no,yes
job,,
student,0.713220,0.286780
retired,0.772085,0.227915
unemployed,0.844973,0.155027
management,0.862444,0.137556
admin.,0.877973,0.122027
self-employed,0.881571,0.118429
unknown,0.881944,0.118056
technician,0.889430,0.110570
services,0.911170,0.088830


In [11]:
# Вплив сімейного стану на схильність до відкриття депозиту
pd.crosstab(df['marital'], df['y'], normalize = 'index')

y,no,yes
marital,,
divorced,0.880545,0.119455
married,0.898765,0.101235
single,0.850508,0.149492


In [12]:
# Середній баланс та середній рівень конверсії для кожного рівня освіти
df['y_binary'] = df['y'].map({'yes': 1, 'no': 0})
df.pivot_table(index = 'education', values = ['y_binary', 'balance'], aggfunc = {'y_binary': 'mean', 'balance': 'mean'}) 

,balance,y_binary
education,,
primary,1250.949934,0.086265
secondary,1154.880786,0.105594
tertiary,1758.416435,0.150064
unknown,1526.754443,0.135703


In [13]:
# Розрахунок конверсії та середнього капіталу за професіями
df.pivot_table(index = 'job', values = ['y_binary', 'balance'], aggfunc = {'y_binary': 'mean', 'balance': 'mean'})

,balance,y_binary
job,,
admin.,1135.838909,0.122027
blue-collar,1078.826654,0.072750
entrepreneur,1521.470074,0.082717
housemaid,1392.395161,0.087903
management,1763.616832,0.137556
retired,1984.215106,0.227915
self-employed,1647.970868,0.118429
services,997.088108,0.088830
student,1388.060768,0.286780


In [14]:
# Середня тривалість успішних розмов для кожного рівня освіти
df[df['y'] == 'yes'].groupby('education')['duration'].mean().sort_values()

education
unknown      463.956349
tertiary     493.369238
secondary    557.884490
primary      631.560068
Name: duration, dtype: float64

In [15]:
# Середня тривалість розмов
df.groupby(df['y'] == 'yes')['duration'].mean()

y
False    221.182806
True     537.294574
Name: duration, dtype: float64

In [16]:
# Середня тривалість розмов за професіями
df[df['y'] == 'yes'].groupby('job')['duration'].mean().sort_values()

job
student          368.200743
retired          460.325581
unknown          476.205882
management       499.620292
admin.           503.114105
technician       539.433333
housemaid        552.688073
unemployed       555.673267
self-employed    584.042781
services         621.804878
entrepreneur     642.991870
blue-collar      675.355932
Name: duration, dtype: float64

In [17]:
# Кількість дзвінків зроблена клієнту за освітою
df.groupby('education')['campaign'].mean().sort_values()

education
secondary    2.700802
tertiary     2.825577
primary      2.834331
unknown      2.849219
Name: campaign, dtype: float64

In [18]:
# Кількість дзвінків зроблена клієнту за професією
df.groupby('job')['campaign'].mean().sort_values()

job
student          2.299574
retired          2.346731
unemployed       2.432080
admin.           2.575324
services         2.718344
entrepreneur     2.799597
blue-collar      2.816995
housemaid        2.820968
self-employed    2.853072
management       2.864348
technician       2.906805
unknown          3.309028
Name: campaign, dtype: float64

In [19]:
# Аналіз досвіду попередніх каипаній у розрізі професій
pd.crosstab(df['job'], df['poutcome'], normalize = 'index')

poutcome,failure,other,success,unknown
job,,,,
admin.,0.124347,0.046606,0.039451,0.789596
blue-collar,0.109741,0.041204,0.015208,0.833847
entrepreneur,0.110289,0.026227,0.014795,0.848689
housemaid,0.078226,0.020968,0.023387,0.877419
management,0.110383,0.041552,0.040918,0.807147
retired,0.101590,0.036661,0.076855,0.784894
self-employed,0.104497,0.038632,0.034832,0.822039
services,0.106885,0.042610,0.020462,0.830043
student,0.124733,0.086354,0.092751,0.696162


In [20]:
# Відсоток конверсії для кожного способу контакту
pd.crosstab(df['contact'], df['y'], normalize = 'index')

y,no,yes
contact,,
cellular,0.850811,0.149189
telephone,0.865795,0.134205
unknown,0.959293,0.040707


In [21]:
df['age'].describe()

count    45211.000000
mean        40.936210
std         10.618762
min         18.000000
25%         33.000000
50%         39.000000
75%         48.000000
max         95.000000
Name: age, dtype: float64

In [22]:
# Сегментація клієнтів за віком для розрахунку конверсії
bins = [18, 30, 40, 50, 60, 100]
labels = ['18-29', '30-39', '40-49', '50-59', '60+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [23]:
df.groupby('age_group')['y_binary'].mean().round(3)*100

C:\Users\PC\AppData\Local\Temp\ipykernel_12832\1294601401.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('age_group')['y_binary'].mean().round(3)*100


age_group
18-29    17.6
30-39    10.6
40-49     9.1
50-59     9.3
60+      33.6
Name: y_binary, dtype: float64

In [24]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,y_binary,age_group
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0,50-59
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0,40-49
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0,30-39
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0,40-49
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0,30-39


In [25]:
# Конвертація та збереження
df.to_csv(r'C:\Users\PC\Desktop\csv\EDA.csv', index=False)